In [1]:
import collections
import pathlib

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

In [2]:
# Download the dataset
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'
dataset_dir = utils.get_file(
    origin=data_url,
    untar=True,
    cache_dir='stack_overflow',
    cache_subdir='')

dataset_dir = pathlib.Path(dataset_dir).parent

6062080/6053168 [==============================] - 158s 26us/step


In [3]:
#display dataset path
list(dataset_dir.iterdir())

[WindowsPath('/tmp/.keras/README.md'),
 WindowsPath('/tmp/.keras/stack_overflow_16k.tar.gz'),
 WindowsPath('/tmp/.keras/test'),
 WindowsPath('/tmp/.keras/train')]

In [4]:
train_dir = dataset_dir/'train'
list(train_dir.iterdir())

[WindowsPath('/tmp/.keras/train/csharp'),
 WindowsPath('/tmp/.keras/train/java'),
 WindowsPath('/tmp/.keras/train/javascript'),
 WindowsPath('/tmp/.keras/train/python')]

In [5]:
# read sample file
sample_file = train_dir/'python/1755.txt'

with open(sample_file) as f:
  print(f.read())


why does this blank program print true x=true.def stupid():.    x=false.stupid().print x



In [6]:
# load dataset
batch_size = 32
seed = 42

raw_train_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)


Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [9]:
print(raw_train_ds)

<BatchDataset shapes: ((None,), (None,)), types: (tf.string, tf.int32)>


In [10]:
#display sample data
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(10):
    print("Question: ", text_batch.numpy()[i])
    print("Label:", label_batch.numpy()[i])

Question:  b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can be easily fixed, please forgive me. my program has a tester class with a main. when i send that to my regularpolygon class, it sends it to the wrong constructor. i have two constructors. 1 without perameters..public regularpolygon().    {.       mynumsides = 5;.       mysidelength = 30;.    }//end default constructor...and my second, with perameters. ..public regularpolygon(int numsides, double sidelength).    {.        mynumsides = numsides;.        mysidelength = sidelength;.    }// end constructor...in my tester class i have these two lines:..regularpolygon shape = new regularpolygon(numsides, sidelength);.        shape.menu();...numsides and sidelength were declared and initialized earlier in the testing class...so what i want to happen, is the tester class sends numsides and sidelength to the second constructor and use it in that class. but it only uses the default con

In [11]:
# The labels are 0, 1, 2 or 3. To check which of these correspond to which string label, 
# you can inspect the class_names property on the dataset:

for i, label in enumerate(raw_train_ds.class_names):
  print("Label", i, "corresponds to", label)


Label 0 corresponds to csharp
Label 1 corresponds to java
Label 2 corresponds to javascript
Label 3 corresponds to python


In [12]:
# Create a validation set.
raw_val_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [13]:
test_dir = dataset_dir/'test'

# Create a test set.
raw_test_ds = utils.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size)


Found 8000 files belonging to 4 classes.


In [14]:
VOCAB_SIZE = 10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')


In [15]:
MAX_SEQUENCE_LENGTH = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)


In [16]:
# Make a text-only dataset (without labels), then call `TextVectorization.adapt`.
train_text = raw_train_ds.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)


In [19]:
print(binary_vectorize_layer)

In [20]:
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return binary_vectorize_layer(text), label


In [21]:
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label


In [22]:
# Retrieve a batch (of 32 reviews and labels) from the dataset.
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print("Question", first_question)
print("Label", first_label)

Question tf.Tensor(b'"what is the difference between these two ways to create an element? var a = document.createelement(\'div\');..a.id = ""mydiv"";...and..var a = document.createelement(\'div\').id = ""mydiv"";...what is the difference between them such that the first one works and the second one doesn\'t?"\n', shape=(), dtype=string)
Label tf.Tensor(2, shape=(), dtype=int32)


In [28]:
label_batch

<tf.Tensor: shape=(32,), dtype=int32, numpy=
array([2, 0, 1, 1, 1, 0, 0, 1, 1, 0, 3, 0, 1, 2, 1, 2, 0, 2, 0, 1, 0, 1,
       0, 0, 2, 1, 1, 0, 0, 0, 1, 1])>

In [26]:
print("'binary' vectorized question:",
      binary_vectorize_text(first_question, first_label)[0])

'binary' vectorized question: tf.Tensor([[1. 1. 0. ... 0. 0. 0.]], shape=(1, 10000), dtype=float32)


In [29]:
first_question

<tf.Tensor: shape=(), dtype=string, numpy=b'"what is the difference between these two ways to create an element? var a = document.createelement(\'div\');..a.id = ""mydiv"";...and..var a = document.createelement(\'div\').id = ""mydiv"";...what is the difference between them such that the first one works and the second one doesn\'t?"\n'>

In [25]:
print("'int' vectorized question:",
      int_vectorize_text(first_question, first_label)[0])


'int' vectorized question: tf.Tensor(
[[ 55   6   2 410 212 229 121 895   4 124  32 245  42   5   1   1   5   1
    1   6   2 410 212 192 318  14   2  98  71 187   8   2 199  71 178   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0


In [33]:
print("1289 ---> ", int_vectorize_layer.get_vocabulary()[245])
print("313 ---> ", int_vectorize_layer.get_vocabulary()[313])
print("Vocabulary size: {}".format(len(int_vectorize_layer.get_vocabulary())))


1289 --->  element
313 --->  source
Vocabulary size: 10000


In [35]:
# As a final preprocessing step, you will apply the TextVectorization 
# layers you created earlier to the training, validation, and test sets:

binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_val_ds = raw_val_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)


In [39]:
print(binary_train_ds)

<MapDataset shapes: ((None, 10000), (None,)), types: (tf.float32, tf.int32)>


In [40]:
# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)


In [41]:
binary_train_ds = configure_dataset(binary_train_ds)
binary_val_ds = configure_dataset(binary_val_ds)
binary_test_ds = configure_dataset(binary_test_ds)

int_train_ds = configure_dataset(int_train_ds)
int_val_ds = configure_dataset(int_val_ds)
int_test_ds = configure_dataset(int_test_ds)


In [42]:
binary_model = tf.keras.Sequential([layers.Dense(4)])

binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

history = binary_model.fit(
    binary_train_ds, validation_data=binary_val_ds, epochs=10)


Epoch 1/10
200/200 [==============================] - 18s 77ms/step - loss: 1.1231 - accuracy: 0.6444 - val_loss: 0.9149 - val_accuracy: 0.7750
Epoch 2/10
200/200 [==============================] - 1s 3ms/step - loss: 0.7803 - accuracy: 0.8191 - val_loss: 0.7497 - val_accuracy: 0.8012
Epoch 3/10
200/200 [==============================] - 1s 3ms/step - loss: 0.6285 - accuracy: 0.8608 - val_loss: 0.6639 - val_accuracy: 0.8175
Epoch 4/10
200/200 [==============================] - 1s 3ms/step - loss: 0.5349 - accuracy: 0.8872 - val_loss: 0.6105 - val_accuracy: 0.8269
Epoch 5/10
200/200 [==============================] - 1s 3ms/step - loss: 0.4688 - accuracy: 0.9025 - val_loss: 0.5739 - val_accuracy: 0.8300
Epoch 6/10
200/200 [==============================] - 1s 3ms/step - loss: 0.4185 - accuracy: 0.9156 - val_loss: 0.5474 - val_accuracy: 0.8338
Epoch 7/10
200/200 [==============================] - 1s 3ms/step - loss: 0.3782 - accuracy: 0.9266 - val_loss: 0.5273 - val_accuracy: 0.8356
Epoc

In [44]:
# Next, you will use the 'int' vectorized layer to build a 1D ConvNet:

def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  return model

In [45]:
# `vocab_size` is `VOCAB_SIZE + 1` since `0` is used additionally for padding.
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=4)
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=5)


Epoch 1/5
200/200 [==============================] - 10s 42ms/step - loss: 1.1477 - accuracy: 0.4980 - val_loss: 0.7673 - val_accuracy: 0.6906
Epoch 2/5
200/200 [==============================] - 5s 26ms/step - loss: 0.6297 - accuracy: 0.7514 - val_loss: 0.5459 - val_accuracy: 0.7944
Epoch 3/5
200/200 [==============================] - 5s 25ms/step - loss: 0.3847 - accuracy: 0.8794 - val_loss: 0.4799 - val_accuracy: 0.8119
Epoch 4/5
200/200 [==============================] - 5s 24ms/step - loss: 0.2143 - accuracy: 0.9478 - val_loss: 0.4788 - val_accuracy: 0.8175
Epoch 5/5
200/200 [==============================] - 5s 25ms/step - loss: 0.1068 - accuracy: 0.9825 - val_loss: 0.5057 - val_accuracy: 0.8144


In [46]:
# Compare the two models:

print("Linear model on binary vectorized data:")
print(binary_model.summary())


Linear model on binary vectorized data:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 40004     
Total params: 40,004
Trainable params: 40,004
Non-trainable params: 0
_________________________________________________________________
None


In [47]:
print("ConvNet model on int vectorized data:")
print(int_model.summary())


ConvNet model on int vectorized data:
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          640064    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          20544     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 660,868
Trainable params: 660,868
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_ds)
int_loss, int_accuracy = int_model.evaluate(int_test_ds)

print("Binary model accuracy: {:2.2%}".format(binary_accuracy))
print("Int model accuracy: {:2.2%}".format(int_accuracy))


250/250 [==============================] - 4s 17ms/step - loss: 0.5233 - accuracy: 0.8079
Binary model accuracy: 81.38%
Int model accuracy: 80.79%


In [49]:
# In the code above, you applied tf.keras.layers.TextVectorization to the 
# dataset before feeding text to the model. If you want to make your model 
# capable of processing raw strings (for example, to simplify deploying it), 
# you can include the TextVectorization layer inside your model.


export_model = tf.keras.Sequential(
    [binary_vectorize_layer, binary_model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print("Accuracy: {:2.2%}".format(binary_accuracy))


250/250 [==============================] - 4s 13ms/step - loss: 0.5193 - accuracy: 0.8138
Accuracy: 81.38%


In [50]:
def get_string_labels(predicted_scores_batch):
  predicted_int_labels = tf.argmax(predicted_scores_batch, axis=1)
  predicted_labels = tf.gather(raw_train_ds.class_names, predicted_int_labels)
  return predicted_labels

In [63]:
inputs = [
    "prayer = 8",  # 'python'
    "debug public static void main(string[] args) {...}",  # 'java'
]
predicted_scores = export_model.predict(inputs)
predicted_labels = get_string_labels(predicted_scores)
for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("Predicted label: ", label.numpy())


Question:  prayer = 8
Predicted label:  b'javascript'
Question:  debug public static void main(string[] args) {...}
Predicted label:  b'java'
